# Build A Simple LLM Application with LangChian and Ollama

## LangSmith
Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. THe best way to do this is with LangSmith.

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

[Ollama](https://ollama.ai/) is one way to easily run inference on macOS.

The instructions [here](https://github.com/jmorganca/ollama?tab=readme-ov-file#ollama) provide details, which we summarize:

• [Download and run](https://ollama.ai/download) the app  
• From command line, fetch a model from this [list of options](https://github.com/jmorganca/ollama): e.g., `ollama pull llama3.1:8b`  
• When the app is running, all models are automatically served on `localhost:11434`  

Stream tokens as they are being generated:

In [3]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1:latest")

for chunk in llm.stream("The first man on the moon was ..."):
    print(chunk, end="|", flush=True)

...|Neil| Armstrong|!| He| stepped| out| of| the| lunar| module| Eagle| and| onto| the| surface| of| the| moon| on| July| |20|,| |196|9|,| famously| declaring| "|That|'s| one| small| step| for| man|,| one| giant| leap| for| mankind|."||

API Reference: [OllamaLLM](https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html)  
Ollama also includes a chat model wrapper that handles formatting conversation turns:

In [6]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.1:latest", format="json")
messages = [
    ("human", "Return a query for the weather in a random location and time of day with two keys: location and time_of_day. Respond using JSON only."),
]
chat_model.invoke(messages).content

'{\n    "location": "Tokyo, Japan",\n    "time_of_day": "12:45 PM"\n}'

API Reference: [ChatOllama](https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html)

## Tool calling

In [8]:
import ollama

response = ollama.chat(
    model='llama3.1',
    messages=[{'role': 'user', 'content': 'What is the weather in Toronto?'}],
    
    # provide a weather checking tool to the model
    tools=[{
        'type': 'function',
        'function': {
            'name': 'get_current_weather',
            'description': 'Get the current weather for a city',
            'parameters': {
                'type': 'object',
                'properties': {
                    'city': {
                        'type': 'string',
                        'description': 'The name of the city',
                    },
                },
                'required': ['city'],
            },
        },
    },
    ],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_current_weather', 'arguments': {'city': 'Toronto'}}}]
